<a href="https://colab.research.google.com/github/bradymiller2310/FantasyFootballDashboard/blob/main/ESPN_FF_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install espn-api
!pip install pandas
!pip install openpyxl

#pip uninstall -y numpy catboost
!pip install numpy==1.24.4  # Safe version compatible with catboost
!pip install catboost --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 266.8 MB/s eta 0:00:00


In [2]:
from espn_api.football import League
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool

In [3]:
league = League(league_id=1292514, year=2024, espn_s2="AECPNWyLvfnlcFfE0Xb73O%2BP5ZU9n1G%2FSnP3ixxvuqSwQRcQe7bEQfBuz3YBNeRluTYofdhX2aTajj0jZSF%2B8i9ewdk8Dwf6p9G215F2FjmI06wPKyhjleL4WzaT3PCiP15FDXe55YNMkQeJXgbsCuK1VHbdQ1Nxidq0JxAHezRn2zAt6WwNtSqrF0k2nNHV33VpJqPmMk4msFmaoCaz0UAp95xxy1YmXyEUnZWgA60HJkbsgYkZ8AaESwFENsPYdUvKYqCwnZLjvgI5varhOTDYekjvokdtaCoxJfzOdGLJ3g%3D%3D", swid="{D68FE5F2-46CE-421C-B248-DDA01360BB34}")
print(league.teams)  #Teams in league

[Team(Fightin' Furries), Team(Quon Solo), Team(Jen-eral ⚔️), Team(Captain Sweatpants), Team(Salmon LipBalm !!), Team(Graham’s Groupie), Team(pop-pop's bible study), Team(Bucktown Bandits), Team(bird gang), Team(bungalicious  💅)]


## **Testing out api functionalities**
*This section contains code chunks that load in various data - used to see what we had access to.*

In [ ]:
for week in range(1, 15):
    print(f"Week {week} Schedule:")
    for matchup in league.scoreboard(week=week):
        print(f"{matchup.home_team} vs {matchup.away_team}")

Week 1 Schedule:
Team(Bucktown Bandits) vs Team(Graham’s Groupie)
Team(bungalicious  💅) vs Team(Fightin' Furries)
Team(bird gang) vs Team(Salmon LipBalm !!)
Team(pop-pop's bible study) vs Team(Quon Solo)
Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
Week 2 Schedule:
Team(Fightin' Furries) vs Team(Salmon LipBalm !!)
Team(Quon Solo) vs Team(Bucktown Bandits)
Team(Captain Sweatpants) vs Team(Graham’s Groupie)
Team(pop-pop's bible study) vs Team(bungalicious  💅)
Team(Jen-eral ⚔️) vs Team(bird gang)
Week 3 Schedule:
Team(Quon Solo) vs Team(Captain Sweatpants)
Team(Fightin' Furries) vs Team(pop-pop's bible study)
Team(Salmon LipBalm !!) vs Team(Jen-eral ⚔️)
Team(Bucktown Bandits) vs Team(bungalicious  💅)
Team(Graham’s Groupie) vs Team(bird gang)
Week 4 Schedule:
Team(pop-pop's bible study) vs Team(Jen-eral ⚔️)
Team(bungalicious  💅) vs Team(Quon Solo)
Team(bird gang) vs Team(Captain Sweatpants)
Team(Bucktown Bandits) vs Team(Fightin' Furries)
Team(Graham’s Groupie) vs Team(Salmon LipBalm !!)


In [ ]:
free_agents = league.free_agents(size=20, position='QB')  # Get top 20 available QBs

print("Available Free Agents (QB):")
for player in free_agents:
    print(f"  - {player.name} ({player.proTeam})")

Available Free Agents (QB):
  - C.J. Stroud (HOU)
  - Tua Tagovailoa (MIA)
  - Anthony Richardson (IND)
  - Aaron Rodgers (None)
  - Kirk Cousins (ATL)
  - Bryce Young (CAR)
  - Michael Penix Jr. (ATL)
  - Trevor Lawrence (JAX)
  - Jameis Winston (CLE)
  - Derek Carr (NO)
  - Justin Fields (NYJ)
  - Joe Flacco (IND)
  - Deshaun Watson (CLE)
  - Cooper Rush (BAL)
  - Daniel Jones (IND)
  - Tom Brady (TB)
  - Mason Rudolph (PIT)
  - Mac Jones (SF)
  - Tommy DeVito (NYG)
  - Aidan O'Connell (LV)


In [ ]:
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    print("Roster:")
    for player in team.roster:
        print(f"  - {player.name} ({player.position})")


Team: Fightin' Furries
Roster:
  - Tyreek Hill (WR)
  - Isiah Pacheco (RB)
  - Joe Mixon (RB)
  - Patrick Mahomes (QB)
  - Dalton Kincaid (TE)
  - Zay Flowers (WR)
  - Jaxon Smith-Njigba (WR)
  - Gus Edwards (RB)
  - Matthew Stafford (QB)
  - Kareem Hunt (RB)
  - Broncos D/ST (D/ST)
  - David Njoku (TE)
  - Caleb Williams (QB)
  - DeAndre Hopkins (WR)
  - Bills D/ST (D/ST)
  - Matthew Wright (K)

Team: Quon Solo
Roster:
  - Justin Jefferson (WR)
  - Kyren Williams (RB)
  - Marvin Harrison Jr. (WR)
  - Mike Evans (WR)
  - D'Andre Swift (RB)
  - Brian Robinson Jr. (RB)
  - Kyler Murray (QB)
  - J.K. Dobbins (RB)
  - Geno Smith (QB)
  - Chris Boswell (K)
  - Romeo Doubs (WR)
  - Mark Andrews (TE)
  - Bears D/ST (D/ST)
  - Courtland Sutton (WR)
  - Josh Downs (WR)
  - Evan Engram (TE)

Team: Jen-eral ⚔️
Roster:
  - Derrick Henry (RB)
  - A.J. Brown (WR)
  - Alvin Kamara (RB)
  - Jalen Hurts (QB)
  - Calvin Ridley (WR)
  - Raheem Mostert (RB)
  - Jets D/ST (D/ST)
  - Pat Freiermuth (TE)
  

In [ ]:
week_number = 1
box_scores = league.box_scores(week=week_number)

for matchup in box_scores:
    print(f"\nMatchup: {matchup.home_team} vs {matchup.away_team}")
    print(f"  Home Score: {matchup.home_score}")
    print(f"  Away Score: {matchup.away_score}")


Matchup: Team(Bucktown Bandits) vs Team(Graham’s Groupie)
  Home Score: 73.0
  Away Score: 96.0

Matchup: Team(bungalicious  💅) vs Team(Fightin' Furries)
  Home Score: 101.0
  Away Score: 94.0

Matchup: Team(bird gang) vs Team(Salmon LipBalm !!)
  Home Score: 59.0
  Away Score: 101.0

Matchup: Team(pop-pop's bible study) vs Team(Quon Solo)
  Home Score: 79.0
  Away Score: 97.0

Matchup: Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
  Home Score: 82.0
  Away Score: 110.0


In [ ]:
# getting injury status of free agents (not on rosters)

free_agents = league.free_agents()
i=0
for player in free_agents:
  player_obj = free_agents[i]
  print(f"Player: {player_obj.name}, Injury Status: {player_obj.injuryStatus}, Injured: {player_obj.injured}")
  i+=1

Player: Ladd McConkey, Injury Status: ACTIVE, Injured: False
Player: Keenan Allen, Injury Status: ACTIVE, Injured: False
Player: Michael Pittman Jr., Injury Status: QUESTIONABLE, Injured: False
Player: Steelers D/ST, Injury Status: [], Injured: False
Player: C.J. Stroud, Injury Status: ACTIVE, Injured: False
Player: Kyle Pitts, Injury Status: ACTIVE, Injured: False
Player: Adam Thielen, Injury Status: ACTIVE, Injured: False
Player: Chris Olave, Injury Status: QUESTIONABLE, Injured: False
Player: Tucker Kraft, Injury Status: ACTIVE, Injured: False
Player: 49ers D/ST, Injury Status: [], Injured: False
Player: Jerome Ford, Injury Status: QUESTIONABLE, Injured: False
Player: Texans D/ST, Injury Status: [], Injured: False
Player: Quentin Johnston, Injury Status: ACTIVE, Injured: False
Player: Tua Tagovailoa, Injury Status: QUESTIONABLE, Injured: False
Player: Nick Chubb, Injury Status: QUESTIONABLE, Injured: False
Player: Anthony Richardson, Injury Status: QUESTIONABLE, Injured: False
Playe

In [ ]:
# getting injury status of players on rosters
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    for player in team.roster:
        print(f"Player: {player.name}, Injury Status: {player.injuryStatus}, Injured: {player.injured}")


Team: Fightin' Furries
Player: Tyreek Hill, Injury Status: QUESTIONABLE, Injured: False
Player: Isiah Pacheco, Injury Status: ACTIVE, Injured: False
Player: Joe Mixon, Injury Status: ACTIVE, Injured: False
Player: Patrick Mahomes, Injury Status: ACTIVE, Injured: False
Player: Dalton Kincaid, Injury Status: ACTIVE, Injured: False
Player: Zay Flowers, Injury Status: QUESTIONABLE, Injured: False
Player: Jaxon Smith-Njigba, Injury Status: ACTIVE, Injured: False
Player: Gus Edwards, Injury Status: ACTIVE, Injured: False
Player: Matthew Stafford, Injury Status: ACTIVE, Injured: False
Player: Kareem Hunt, Injury Status: ACTIVE, Injured: False
Player: Broncos D/ST, Injury Status: NORMAL, Injured: False
Player: David Njoku, Injury Status: QUESTIONABLE, Injured: False
Player: Caleb Williams, Injury Status: QUESTIONABLE, Injured: False
Player: DeAndre Hopkins, Injury Status: ACTIVE, Injured: False
Player: Bills D/ST, Injury Status: NORMAL, Injured: False
Player: Matthew Wright, Injury Status: AC

In [ ]:
# getting starting roster for each week

for team in league.teams:
    print(f"\nTeam: {team.team_name} (Week {week})")

    # Get the team's roster for the given week
    for player in team.roster:
        #if player.lineupSlot != "BE" and player.lineupSlot != "IR":  # Exclude Bench and IR players
            print(f"  - {player.name} ({player.lineupSlot})")



Team: Fightin' Furries (Week 14)
  - Tyreek Hill (WR)
  - Isiah Pacheco (RB)
  - Joe Mixon (BE)
  - Patrick Mahomes (QB)
  - Dalton Kincaid (BE)
  - Zay Flowers (BE)
  - Jaxon Smith-Njigba (RB/WR/TE)
  - Gus Edwards (BE)
  - Matthew Stafford (BE)
  - Kareem Hunt (RB)
  - Broncos D/ST (BE)
  - David Njoku (TE)
  - Caleb Williams (BE)
  - DeAndre Hopkins (WR)
  - Bills D/ST (D/ST)
  - Matthew Wright (K)

Team: Quon Solo (Week 14)
  - Justin Jefferson (WR)
  - Kyren Williams (RB)
  - Marvin Harrison Jr. (BE)
  - Mike Evans (BE)
  - D'Andre Swift (RB)
  - Brian Robinson Jr. (RB/WR/TE)
  - Kyler Murray (QB)
  - J.K. Dobbins (BE)
  - Geno Smith (BE)
  - Chris Boswell (K)
  - Romeo Doubs (BE)
  - Mark Andrews (TE)
  - Bears D/ST (D/ST)
  - Courtland Sutton (WR)
  - Josh Downs (BE)
  - Evan Engram (BE)

Team: Jen-eral ⚔️ (Week 14)
  - Derrick Henry (RB)
  - A.J. Brown (WR)
  - Alvin Kamara (RB)
  - Jalen Hurts (QB)
  - Calvin Ridley (WR)
  - Raheem Mostert (BE)
  - Jets D/ST (D/ST)
  - Pat Fr

## **Getting the fantasy team rosters**
*Will use to join to player data to maintain team assigments*

In [4]:
# Define the range of weeks
weeks = range(1, 19)  # Weeks 1 to 18

# Create an empty list to store data
all_weeks_roster = []

# Loop through each week and get the roster data
for week in weeks:
    week_data = []  # Temporary storage for current week's data

    for team in league.teams:
        team_name = team.team_name  # Fantasy Team Name

        for player in team.roster:
            week_data.append({
                "cur_Fteam": team_name,
                "week": week,
                "espn_id": player.playerId,  # ESPN Unique Player ID
                "cur_roster_slot": player.position if player.lineupSlot != "BE" else "Bench"
            })

    # If no data is returned for this week, stop iterating
    if not week_data:
        print(f"No data found for Week {week}. Stopping iteration.")
        break  # Stop looping when a week has no data

    # Add this week's data to the master list
    all_weeks_roster.extend(week_data)

# Convert to DataFrame
weekly_rosters = pd.DataFrame(all_weeks_roster)

cur_roster = weekly_rosters[weekly_rosters['week'] == weekly_rosters['week'].max()].drop('week', axis=1)
print(cur_roster)

             cur_Fteam  espn_id cur_roster_slot
2720  Fightin' Furries  3116406              WR
2721  Fightin' Furries  4361529              RB
2722  Fightin' Furries  3116385           Bench
2723  Fightin' Furries  3139477              QB
2724  Fightin' Furries  4385690           Bench
...                ...      ...             ...
2875   bungalicious  💅  4243537           Bench
2876   bungalicious  💅  4689936               K
2877   bungalicious  💅  3912547              QB
2878   bungalicious  💅   -16011            D/ST
2879   bungalicious  💅  4426385              RB

[160 rows x 3 columns]


### **Getting statistical data from xlsx files**

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [6]:
file_path = "/content/drive/My Drive/ff_data.xlsx"

# Load specific sheets
season_stats_all = pd.read_excel(file_path, sheet_name="Season", engine="openpyxl")
weekly_stats_all = pd.read_excel(file_path, sheet_name="Weekly", engine="openpyxl")

# Display first few rows
print(season_stats_all.head())
print(weekly_stats_all.head())

   season   player_id player_display_name team position  games_played  \
0    2021  00-0019596           Tom Brady   TB       QB          20.0   
1    2021  00-0022924  Ben Roethlisberger  PIT       QB          18.0   
2    2021  00-0023459       Aaron Rodgers   GB       QB          17.0   
3    2021  00-0023682    Ryan Fitzpatrick  WAS       QB           1.0   
4    2021  00-0024243      Marcedes Lewis   GB       TE          14.0   

   completions  attempts  comp%  passing_yards  ...  total_st_tds  st_tds_pg  \
0        573.0     847.0  67.65         6187.0  ...           NaN        NaN   
1        448.0     693.0  64.65         4170.0  ...           NaN        NaN   
2        386.0     560.0  68.93         4340.0  ...           NaN        NaN   
3          3.0       6.0  50.00           13.0  ...           NaN        NaN   
4          0.0       0.0    NaN            0.0  ...           NaN        NaN   

   total_fp  fp_pg  espn_id                name   age  height  weight  \
0      

## **Joining current fantasy team roster data with weekly and season data**

### *Season data*

In [7]:
season_stats_all = pd.merge(season_stats_all, cur_roster, on="espn_id", how="left")

season_stats_all.loc[
    (season_stats_all['season'] == 2024) &
    (season_stats_all['cur_Fteam'] != 'FA') &
    (season_stats_all['cur_roster_slot'] != 'FA'),
    ['cur_Fteam', 'cur_roster_slot']
] = 'FA'

In [8]:
#print(season_stats_all.head(10))

   season   player_id player_display_name      team position  games_played  \
0    2021  00-0019596           Tom Brady        TB       QB          20.0   
1    2021  00-0022924  Ben Roethlisberger       PIT       QB          18.0   
2    2021  00-0023459       Aaron Rodgers        GB       QB          17.0   
3    2021  00-0023682    Ryan Fitzpatrick       WAS       QB           1.0   
4    2021  00-0024243      Marcedes Lewis        GB       TE          14.0   
5    2021  00-0025394     Adrian Peterson  TEN, SEA       RB           4.0   
6    2021  00-0026035      Danny Amendola       HOU       WR           8.0   
7    2021  00-0026143           Matt Ryan       ATL       QB          17.0   
8    2021  00-0026158          Joe Flacco       NYJ       QB           2.0   
9    2021  00-0026189      DeSean Jackson    LA, LV       WR          16.0   

   completions  attempts  comp%  passing_yards  ...  total_fp  fp_pg  espn_id  \
0        573.0     847.0  67.65         6187.0  ...       Na

### *Weekly data*

In [8]:
weekly_stats_all = pd.merge(weekly_stats_all, weekly_rosters, on=["espn_id", "week"], how="left")

weekly_stats_all.loc[
    (weekly_stats_all['season'] == 2024) &
    (weekly_stats_all['cur_Fteam'] != 'FA') &
    (weekly_stats_all['cur_roster_slot'] != 'FA'),
    ['cur_Fteam', 'cur_roster_slot']
] = 'FA'

In [11]:
#print(weekly_stats_all.head(10))

    player_id player_name player_display_name position position_group  \
0  00-0019596     T.Brady           Tom Brady       QB             QB   
1  00-0019596     T.Brady           Tom Brady       QB             QB   
2  00-0019596     T.Brady           Tom Brady       QB             QB   
3  00-0019596     T.Brady           Tom Brady       QB             QB   
4  00-0019596     T.Brady           Tom Brady       QB             QB   
5  00-0019596     T.Brady           Tom Brady       QB             QB   
6  00-0019596     T.Brady           Tom Brady       QB             QB   
7  00-0019596     T.Brady           Tom Brady       QB             QB   
8  00-0019596     T.Brady           Tom Brady       QB             QB   
9  00-0019596     T.Brady           Tom Brady       QB             QB   

                                        headshot_url recent_team  season  \
0  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
1  https://static.www.nfl.com/image/private/

# **Getting matchup data**

In [9]:
total_weeks = league.settings.reg_season_count  # Regular season length

# Initialize records dict
team_records = {team.team_name: {'W': 0, 'L': 0} for team in league.teams}

# Store all weekly matchups and records
matchups_all_weeks = []

# Loop through each week
for week in range(1, total_weeks + 1):
    scoreboard = league.scoreboard(week=week)

    for matchup in scoreboard:
        home = matchup.home_team.team_name
        away = matchup.away_team.team_name
        home_score = matchup.home_score
        away_score = matchup.away_score

        # Determine winner and update records
        if home_score > away_score:
            team_records[home]['W'] += 1
            team_records[away]['L'] += 1
        elif home_score < away_score:
            team_records[away]['W'] += 1
            team_records[home]['L'] += 1
        else:
            # Tie case (optional)
            pass

        # Record after this week
        home_record = f"{team_records[home]['W']}-{team_records[home]['L']}"
        away_record = f"{team_records[away]['W']}-{team_records[away]['L']}"

        matchups_all_weeks.append({
            'week': week,
            'home_team': home,
            'away_team': away,
            'home_score': home_score,
            'away_score': away_score,
            'home_record': home_record,
            'away_record': away_record
        })

# Convert to DataFrame
weekly_matchup_data = pd.DataFrame(matchups_all_weeks)

# Display the result
print(weekly_matchup_data)

    week              home_team              away_team  home_score  \
0      1       Bucktown Bandits       Graham’s Groupie        73.0   
1      1        bungalicious  💅       Fightin' Furries       101.0   
2      1              bird gang      Salmon LipBalm !!        59.0   
3      1  pop-pop's bible study              Quon Solo        79.0   
4      1            Jen-eral ⚔️     Captain Sweatpants        82.0   
..   ...                    ...                    ...         ...   
65    14       Graham’s Groupie       Bucktown Bandits       143.0   
66    14      Salmon LipBalm !!       Fightin' Furries       115.0   
67    14     Captain Sweatpants              Quon Solo        88.0   
68    14            Jen-eral ⚔️  pop-pop's bible study        68.0   
69    14              bird gang        bungalicious  💅        81.0   

    away_score home_record away_record  
0         96.0         0-1         1-0  
1         94.0         1-0         0-1  
2        101.0         0-1         1

# **Getting injury status**
PROBABLY CAN'T USE B/C ONLY HAVE 2024 DATA

In [ ]:
all_players = []
current_week = league.current_week

# 1. Rostered players from each team
for team in league.teams:
    for player in team.roster:
        all_players.append({
            "week": current_week,
            "team_owner": team.team_name,
            "name": player.name,
            "position": player.position,
            "espn_id": player.playerId,
            "injured": player.injured,
            "injury_status": player.injuryStatus,
            "rostered": True
        })

# 2. Free agents (unrostered players)
for player in league.free_agents(size=1000):  # you can adjust the size to pull more players
    all_players.append({
        "week": current_week,
        "team_owner": "Free Agent",
        "name": player.name,
        "position": player.position,
        "espn_id": player.playerId,
        "injured": player.injured,
        "injury_status": player.injuryStatus,
        "rostered": False
    })

# Optional: Convert to pandas DataFrame
import pandas as pd
injury_status = pd.DataFrame(all_players)

# Display or export
print(injury_status.head(10))

print(injury_status[injury_status["team_owner"] == "Free Agent"].head(10))

   week        team_owner                name position  espn_id  injured  \
0    17  Fightin' Furries         Tyreek Hill       WR  3116406    False   
1    17  Fightin' Furries       Isiah Pacheco       RB  4361529    False   
2    17  Fightin' Furries           Joe Mixon       RB  3116385    False   
3    17  Fightin' Furries     Patrick Mahomes       QB  3139477    False   
4    17  Fightin' Furries      Dalton Kincaid       TE  4385690    False   
5    17  Fightin' Furries         Zay Flowers       WR  4429615    False   
6    17  Fightin' Furries  Jaxon Smith-Njigba       WR  4430878    False   
7    17  Fightin' Furries         Gus Edwards       RB  3051926    False   
8    17  Fightin' Furries    Matthew Stafford       QB    12483    False   
9    17  Fightin' Furries         Kareem Hunt       RB  3059915    False   

  injury_status  rostered  
0  QUESTIONABLE      True  
1        ACTIVE      True  
2        ACTIVE      True  
3        ACTIVE      True  
4        ACTIVE      Tr

# **QB CatBoost Model**

## **Feature Engineering**
*Need to create features that can be used in both train and test sets since we can't use rush yards, pass yards (any game specific stats) because they have not occurred and need to use like rolling features or averages*

In [ ]:
df_qb = weekly_stats_all[weekly_stats_all['position'] == 'QB'].copy()

# -------------------------------
# STEP 2: Sort for time-series ops
# -------------------------------
df_qb = df_qb.sort_values(by=['player_id', 'season', 'week'])

# -------------------------------
# STEP 3: Lag + Rolling Features (3-game window)
# -------------------------------
rolling_cols = [
    'passing_yards', 'passing_tds', 'interceptions', 'passing_epa',
    'rushing_yards', 'rushing_tds', 'completions', 'attempts',
    'fantasy_points_ppr', 'passing_air_yards', 'passing_yards_after_catch',
    'rushing_fumbles_lost', 'rushing_epa', 'rushing_2pt_conversions', 'passing_2pt_conversions'
]

for col in rolling_cols:
    df_qb[f'{col}_last3'] = df_qb.groupby('player_id')[col].shift(1).rolling(3, min_periods = 1).mean()

# -------------------------------
# STEP 4: Expanding (season-to-date) averages
# -------------------------------
expanding_cols = ['passing_yards', 'passing_tds', 'interceptions', 'rushing_yards', 'rushing_tds', 'fantasy_points_ppr'
]

for col in expanding_cols:
    df_qb[col] = pd.to_numeric(df_qb[col], errors='coerce')

for col in expanding_cols:
    df_qb[f'{col}_season_avg'] = (
        df_qb.groupby(['player_id', 'season'])[col]
        .transform(lambda x: x.shift(1).expanding().mean())
    )

df_qb['completions'] = pd.to_numeric(df_qb['completions'], errors='coerce')
df_qb['attempts'] = pd.to_numeric(df_qb['attempts'], errors='coerce')


df_qb['yards_per_attempt'] = df_qb['passing_yards'] / df_qb['attempts'].replace(0, pd.NA)

df_qb['comp%'] = (
    (df_qb['completions'] / df_qb['attempts']) * 100
).replace([pd.NA, np.inf, -np.inf], np.nan)

df_qb['comp%'] = pd.to_numeric(df_qb['comp%'], errors='coerce')

df_qb['yards_per_attempt'] = pd.to_numeric(df_qb['yards_per_attempt'], errors='coerce')

print(df_qb.head(10))

In [11]:
# -------------------------------
# STEP 5: Efficiency metrics
# -------------------------------
df_qb['pass_attempts_shifted'] = df_qb.groupby('player_id')['attempts'].shift(1)

df_qb['comp_pct_last3'] = df_qb.groupby('player_id')['comp%'].shift(1).rolling(3,min_periods = 1).mean()
df_qb['yards_per_attempt_last3'] = df_qb.groupby('player_id')['yards_per_attempt'].shift(1).rolling(3,min_periods = 1).mean()
df_qb['td_rate_last3'] = df_qb.groupby('player_id')['passing_tds'].shift(1).rolling(3,min_periods = 1).sum() / df_qb['pass_attempts_shifted']
df_qb['int_rate_last3'] = df_qb.groupby('player_id')['interceptions'].shift(1).rolling(3,min_periods = 1).sum() / df_qb['pass_attempts_shifted']

# -------------------------------
# STEP 6: Advanced metrics
# -------------------------------
advanced = ['pacr', 'dakota']
for col in advanced:
    df_qb[f'{col}_trend'] = df_qb.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 7: Opponent Defense (already lagged by design)
# -------------------------------
opponent_cols = ['opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg', 'opp_sacks_pg', 'opp_fumbles_pg']
for col in opponent_cols:
    df_qb[f'{col}_trend'] = df_qb.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 8: Clean up NA rows (from shifting/rolling)
# -------------------------------
#df_qb = df_qb.dropna()

In [ ]:
#print(df_qb.head(10))

***Getting list of columns to select from***

In [42]:
#print(df_qb.columns.tolist())

['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'recent_team', 'season', 'week', 'season_type', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr', 'half_PPR', 'depth_pos', 'offense_snaps', 'offense_pct', 'opp_avg_ypg_al

In [28]:
TARGET_COL = 'fantasy_points_ppr'

train_df = df_qb[
    (df_qb[TARGET_COL].notna()) &
    (
        (df_qb['season'] < 2024) |
        ((df_qb['season'] == 2024) & (df_qb['week'] < 14))
    )
]

test_df = df_qb[(df_qb['season'] == 2024) & (df_qb['week'] == 14)]


feature_cols = [
    col for col in df_qb.columns
    if col.endswith('_last3') or col.endswith('_trend') or col.endswith('_season_avg')
] + ['player_display_name','season', 'week', 'opponent_team']


X_train = train_df[feature_cols]
y_train = train_df[TARGET_COL]

X_test = test_df[feature_cols]

# for RMSE testing purposes on the test set
y_test = test_df[TARGET_COL]

In [32]:
categorical_cols = ['opponent_team', 'season', 'week', 'player_display_name']

# Create Pool objects
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_cols)
test_pool = Pool(data=X_test, cat_features=categorical_cols)

# Initialize the CatBoost model
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50  # or 0 to suppress output
)

# Train
model.fit(train_pool)

0:	learn: 8.7890826	total: 3.1ms	remaining: 617ms
50:	learn: 6.5642620	total: 408ms	remaining: 1.19s
100:	learn: 6.1392943	total: 788ms	remaining: 773ms
150:	learn: 5.6906417	total: 1.18s	remaining: 382ms
199:	learn: 5.2599408	total: 1.55s	remaining: 0us


In [33]:
y_pred = model.predict(test_pool)

# Attach predictions to test set for review
QB_test_pred = X_test.copy()
QB_test_pred['predicted_fantasy_points'] = y_pred

print(QB_test_pred[['season', 'week', 'opponent_team', 'predicted_fantasy_points', 'player_display_name']])

       season  week opponent_team  predicted_fantasy_points  \
16904    2024    14           MIA                 15.512667   
16933    2024    14           BUF                 17.084880   
16953    2024    14           CLE                 14.279731   
16971    2024    14           MIN                 14.486726   
17095    2024    14           ARI                 17.443539   
17141    2024    14           NYG                 14.133496   
17192    2024    14           PIT                 12.461764   
17272    2024    14            KC                 11.199089   
17368    2024    14           CHI                 12.018220   
17510    2024    14            GB                 17.552149   
17624    2024    14           ATL                  8.843295   
17781    2024    14           CIN                 12.940275   
17869    2024    14           LAC                 19.112748   
18439    2024    14            LV                 17.575620   
18457    2024    14            LA                 19.01

### *old*

In [ ]:
train_qb = weekly_stats_all[
    ((weekly_stats_all['season'].astype(int) < 2024) |
     ((weekly_stats_all['season'].astype(int) == 2024) & (weekly_stats_all['week'].astype(int) < 14))) &
    (weekly_stats_all['position'] == 'QB')
].copy()

# Test: only QBs from Week 14 of 2024
test_qb = weekly_stats_all[
    (weekly_stats_all['season'].astype(int) == 2024) &
    (weekly_stats_all['week'].astype(int) == 14) &
    (weekly_stats_all['position'] == 'QB')
].copy()

test_qb = test_qb.dropna(subset=["fantasy_points"])

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.24.4
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, r2_score

**Defining features to be used**

In [ ]:
categorical_features = ['team', 'opponent_team']  # these are column names, not indices
numerical_features = features = [
    'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',
    'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost',
    'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs',
    'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota',
    'carries', 'rushing_yards', 'rushing_tds',
    'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
    'rushing_epa', 'rushing_2pt_conversions',
    'racr', 'wopr',
    'offense_snaps', 'offense_pct',
    'opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg',
    'opp_fumbles_pg', 'opp_df_tds_pg', 'opp_sacks_pg', 'age', 'week', 'season'
] # adjust as needed

features = categorical_features + numerical_features
target = 'fantasy_points'

**Creating test and train sets**

In [ ]:
X_train = train_qb[features]
y_train = train_qb[target]

X_test = test_qb[features]
y_test = test_qb[target]

In [ ]:
# Drop rows where the target is NaN
train_mask = ~y_train.isna()
X_train = X_train[train_mask]
y_train = y_train[train_mask]

test_mask = ~y_test.isna()
X_test = X_test[test_mask]
y_test = y_test[test_mask]

**Training the model**

In [ ]:
for col in categorical_features:
    X_train[col] = X_train[col].fillna("NA").astype(str)
    X_test[col] = X_test[col].fillna("NA").astype(str)

In [ ]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, y_test, cat_features=categorical_features)

# 7. Train the model
model = CatBoostRegressor(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50
)

model.fit(train_pool)

0:	learn: 8.6098981	total: 5.42ms	remaining: 1.62s
50:	learn: 2.1120422	total: 264ms	remaining: 1.29s
100:	learn: 1.1098378	total: 517ms	remaining: 1.02s
150:	learn: 0.8242850	total: 780ms	remaining: 769ms
200:	learn: 0.6784543	total: 1.02s	remaining: 504ms
250:	learn: 0.5612642	total: 1.28s	remaining: 249ms
299:	learn: 0.4891954	total: 1.51s	remaining: 0us


**Testing and evaluating the model**

In [ ]:
# 8. Predict and evaluate
y_pred = model.predict(test_pool)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE on Week 14 of 2024: {mae:.2f}")
print(f"R² Score: {r2:.3f}")

# 9. Attach predictions to test data (optional)
test_qb['predicted_fp'] = y_pred

MAE on Week 14 of 2024: 0.67
R² Score: 0.972


In [ ]:
qb_pred_results = test_qb[[
    'player_display_name',
    'position',
    'team',
    'fantasy_points'
]].copy()

qb_pred_results['proj_fp'] = test_qb['predicted_fp']

# (Optional) Sort by projected points
qb_pred_results = qb_pred_results.sort_values(by='proj_fp', ascending=False)

# Show top rows
print(qb_pred_results)

      player_display_name position team  fantasy_points    proj_fp
18457          Josh Allen       QB  NaN           51.88  42.036743
18484         Sam Darnold       QB  NaN           34.58  33.663590
19480          Joe Burrow       QB  NaN           24.56  25.283274
19359         Jalen Hurts       QB  NaN           24.22  24.771326
19093      Tua Tagovailoa       QB  NaN           23.54  24.391290
20581         Brock Purdy       QB  NaN           22.10  22.612483
17510          Jared Goff       QB  NaN           22.02  22.472872
16933    Matthew Stafford       QB  NaN           20.80  21.276920
18439      Baker Mayfield       QB  NaN           19.00  19.034256
18626        Kyler Murray       QB  NaN           17.96  17.665650
16904       Aaron Rodgers       QB  NaN           17.56  17.589008
16953      Russell Wilson       QB  NaN           16.02  16.184173
18952           Drew Lock       QB  NaN           14.98  15.340196
17192      Jameis Winston       QB  NaN           15.28  15.07

# **RB CatBoost Model**

In [40]:
df_rb = weekly_stats_all[weekly_stats_all['position'] == 'RB'].copy()

# -------------------------------
# STEP 2: Sort for time-series ops
# -------------------------------
df_rb = df_rb.sort_values(by=['player_id', 'season', 'week'])

# -------------------------------
# STEP 3: Lag + Rolling Features (3-game window)
# -------------------------------
rolling_cols = [
    'rushing_yards', 'rushing_tds',
    'fantasy_points_ppr', 'carries',
    'rushing_fumbles_lost', 'rushing_epa', 'rushing_2pt_conversions', 'passing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_epa', 'receiving_2pt_conversions', 'target_share', 'offense_snaps', 'offense_pct', 'air_yards_share'
    ]

for col in rolling_cols:
    df_rb[f'{col}_last3'] = df_rb.groupby('player_id')[col].shift(1).rolling(3, min_periods = 1).mean()

# -------------------------------
# STEP 4: Expanding (season-to-date) averages
# -------------------------------
expanding_cols = ['rushing_yards', 'rushing_tds', 'fantasy_points_ppr'
]

for col in expanding_cols:
    df_rb[col] = pd.to_numeric(df_rb[col], errors='coerce')

for col in expanding_cols:
    df_rb[f'{col}_season_avg'] = (
        df_rb.groupby(['player_id', 'season'])[col]
        .transform(lambda x: x.shift(1).expanding().mean())
    )


df_rb['carries'] = pd.to_numeric(df_rb['carries'], errors='coerce')


df_rb['yards_per_attempt'] = df_rb['rushing_yards'] / df_rb['carries'].replace(0, pd.NA)
df_rb['yards_per_attempt'] = pd.to_numeric(df_rb['yards_per_attempt'], errors='coerce')

print(df_rb.head(10))

      player_id player_name player_display_name position position_group  \
68   00-0025394  A.Peterson     Adrian Peterson       RB             RB   
69   00-0025394  A.Peterson     Adrian Peterson       RB             RB   
70   00-0025394  A.Peterson     Adrian Peterson       RB             RB   
71   00-0025394  A.Peterson     Adrian Peterson       RB             RB   
266  00-0027966    M.Ingram         Mark Ingram       RB             RB   
267  00-0027966    M.Ingram         Mark Ingram       RB             RB   
268  00-0027966    M.Ingram         Mark Ingram       RB             RB   
269  00-0027966    M.Ingram         Mark Ingram       RB             RB   
270  00-0027966    M.Ingram         Mark Ingram       RB             RB   
271  00-0027966    M.Ingram         Mark Ingram       RB             RB   

                                          headshot_url recent_team  season  \
68   https://static.www.nfl.com/image/private/f_aut...         TEN    2021   
69   https://stati

In [41]:
# -------------------------------
# STEP 5: Efficiency metrics
# -------------------------------
df_rb['carries_shifted'] = df_rb.groupby('player_id')['carries'].shift(1)
df_rb['receptions_shifted'] = df_rb.groupby('player_id')['receptions'].shift(1)


df_rb['yards_per_attempt_last3'] = df_rb.groupby('player_id')['yards_per_attempt'].shift(1).rolling(3,min_periods = 1).mean()
df_rb['rush_td_rate_last3'] = df_rb.groupby('player_id')['rushing_tds'].shift(1).rolling(3,min_periods = 1).sum() / df_rb['carries_shifted']
df_rb['rec_td_rate_last3'] = df_rb.groupby('player_id')['receiving_tds'].shift(1).rolling(3,min_periods = 1).sum() / df_rb['receptions_shifted']

# -------------------------------
# STEP 6: Advanced metrics
# -------------------------------
advanced = ['wopr', 'racr']
for col in advanced:
    df_rb[f'{col}_trend'] = df_rb.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 7: Opponent Defense (already lagged by design)
# -------------------------------
opponent_cols = ['opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg', 'opp_sacks_pg', 'opp_fumbles_pg']
for col in opponent_cols:
    df_rb[f'{col}_trend'] = df_rb.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 8: Clean up NA rows (from shifting/rolling)
# -------------------------------
#df_qb = df_qb.dropna()

In [42]:
TARGET_COL = 'fantasy_points_ppr'

train_df = df_rb[
    (df_rb[TARGET_COL].notna()) &
    (
        (df_rb['season'] < 2024) |
        ((df_rb['season'] == 2024) & (df_rb['week'] < 14))
    )
]

test_df = df_rb[(df_rb['season'] == 2024) & (df_rb['week'] == 14)]


feature_cols = [
    col for col in df_rb.columns
    if col.endswith('_last3') or col.endswith('_trend') or col.endswith('_season_avg')
] + ['player_display_name','season', 'week', 'opponent_team']


X_train = train_df[feature_cols]
y_train = train_df[TARGET_COL]

X_test = test_df[feature_cols]

# for RMSE testing purposes on the test set
y_test = test_df[TARGET_COL]

In [43]:
categorical_cols = ['opponent_team', 'season', 'week', 'player_display_name']

# Create Pool objects
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_cols)
test_pool = Pool(data=X_test, cat_features=categorical_cols)

# Initialize the CatBoost model
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50  # or 0 to suppress output
)

# Train
model.fit(train_pool)

0:	learn: 7.6943307	total: 11.7ms	remaining: 2.32s
50:	learn: 6.1574634	total: 482ms	remaining: 1.41s
100:	learn: 5.8774601	total: 974ms	remaining: 954ms
150:	learn: 5.6134295	total: 1.44s	remaining: 466ms
199:	learn: 5.3807481	total: 1.9s	remaining: 0us


In [44]:
y_pred = model.predict(test_pool)

# Attach predictions to test set for review
RB_test_pred = X_test.copy()
RB_test_pred['predicted_fantasy_points'] = y_pred

print(RB_test_pred[['season', 'week', 'opponent_team', 'predicted_fantasy_points', 'player_display_name']])

       season  week opponent_team  predicted_fantasy_points  \
17108    2024    14           CLE                  6.798921   
17290    2024    14            TB                  9.527467   
17470    2024    14           CIN                  5.278855   
17603    2024    14           ATL                 14.132633   
17675    2024    14           LAC                  4.881373   
...       ...   ...           ...                       ...   
22051    2024    14           BUF                  4.569263   
22104    2024    14           MIA                  5.450039   
22117    2024    14           MIA                  5.344833   
22222    2024    14           NYJ                  3.945262   
22444    2024    14           SEA                  5.133767   

         player_display_name  
17108  Cordarrelle Patterson  
17290         Ameer Abdullah  
17470        Ezekiel Elliott  
17603            Aaron Jones  
17675          Samaje Perine  
...                      ...  
22051            Blake Cor

In [46]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Actuals and predictions
y_true = y_test
y_pred = RB_test_pred['predicted_fantasy_points']

# RMSE and MAE
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")

✅ RMSE: 51.73
✅ MAE: 5.15


# **WR CatBoost Model**

In [64]:
df_wr = weekly_stats_all[weekly_stats_all['position'] == 'WR'].copy()

# -------------------------------
# STEP 2: Sort for time-series ops
# -------------------------------
df_wr = df_wr.sort_values(by=['player_id', 'season', 'week'])

# -------------------------------
# STEP 3: Lag + Rolling Features (3-game window)
# -------------------------------
rolling_cols = [
    'rushing_yards', 'rushing_tds',
    'fantasy_points_ppr', 'carries',
    'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_epa', 'receiving_2pt_conversions', 'target_share', 'offense_snaps', 'offense_pct', 'air_yards_share'
    ]

for col in rolling_cols:
    df_wr[f'{col}_last3'] = df_wr.groupby('player_id')[col].shift(1).rolling(3, min_periods = 1).mean()

# -------------------------------
# STEP 4: Expanding (season-to-date) averages
# -------------------------------
expanding_cols = ['receiving_yards', 'receiving_tds', 'fantasy_points_ppr'
]

for col in expanding_cols:
    df_wr[col] = pd.to_numeric(df_wr[col], errors='coerce')

for col in expanding_cols:
    df_wr[f'{col}_season_avg'] = (
        df_wr.groupby(['player_id', 'season'])[col]
        .transform(lambda x: x.shift(1).expanding().mean())
    )


df_wr['receiving_yards'] = pd.to_numeric(df_wr['receiving_yards'], errors='coerce')
df_wr['receptions'] = pd.to_numeric(df_wr['receptions'], errors='coerce')


df_wr['yards_per_catch'] = df_wr['receiving_yards'] / df_wr['receptions'].replace(0, pd.NA)
df_wr['yards_per_catch'] = pd.to_numeric(df_wr['yards_per_catch'], errors='coerce')



print(df_wr.head(10))

      player_id player_name player_display_name position position_group  \
72   00-0026035  D.Amendola      Danny Amendola       WR             WR   
73   00-0026035  D.Amendola      Danny Amendola       WR             WR   
74   00-0026035  D.Amendola      Danny Amendola       WR             WR   
75   00-0026035  D.Amendola      Danny Amendola       WR             WR   
76   00-0026035  D.Amendola      Danny Amendola       WR             WR   
77   00-0026035  D.Amendola      Danny Amendola       WR             WR   
78   00-0026035  D.Amendola      Danny Amendola       WR             WR   
79   00-0026035  D.Amendola      Danny Amendola       WR             WR   
99   00-0026189   D.Jackson      DeSean Jackson       WR             WR   
100  00-0026189   D.Jackson      DeSean Jackson       WR             WR   

                                          headshot_url recent_team  season  \
72   https://static.www.nfl.com/image/private/f_aut...         HOU    2021   
73   https://stati

In [66]:
# -------------------------------
# STEP 5: Efficiency metrics
# -------------------------------
df_wr['carries_shifted'] = df_wr.groupby('player_id')['carries'].shift(1)
df_wr['receptions_shifted'] = df_wr.groupby('player_id')['receptions'].shift(1)


df_wr['yards_per_reception_last3'] = df_wr.groupby('player_id')['yards_per_catch'].shift(1).rolling(3,min_periods = 1).mean()
df_wr['rec_td_rate_last3'] = df_wr.groupby('player_id')['receiving_tds'].shift(1).rolling(3,min_periods = 1).sum() / df_rb['receptions_shifted']

# -------------------------------
# STEP 6: Advanced metrics
# -------------------------------
advanced = ['wopr', 'racr']
for col in advanced:
    df_wr[f'{col}_trend'] = df_wr.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 7: Opponent Defense (already lagged by design)
# -------------------------------
opponent_cols = ['opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg', 'opp_sacks_pg', 'opp_fumbles_pg']
for col in opponent_cols:
    df_wr[f'{col}_trend'] = df_wr.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

In [75]:
TARGET_COL = 'fantasy_points_ppr'

train_df = df_wr[
    (df_wr[TARGET_COL].notna()) &
    (
        (df_wr['season'] < 2024) |
        ((df_wr['season'] == 2024) & (df_wr['week'] < 14))
    )
]

test_df = df_wr[(df_wr['season'] == 2024) & (df_wr['week'] == 14)]


feature_cols = [
    col for col in df_wr.columns
    if col.endswith('_last3') or col.endswith('_trend') or col.endswith('_season_avg')
] + ['player_display_name','season', 'week', 'opponent_team']


X_train = train_df[feature_cols]
y_train = train_df[TARGET_COL]

X_test = test_df[feature_cols]

# for RMSE testing purposes on the test set
y_test = test_df[TARGET_COL]

In [76]:
categorical_cols = ['opponent_team', 'season', 'week', 'player_display_name']

# Create Pool objects
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_cols)
test_pool = Pool(data=X_test, cat_features=categorical_cols)

# Initialize the CatBoost model
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50  # or 0 to suppress output
)

# Train
model.fit(train_pool)

0:	learn: 7.5687522	total: 13.3ms	remaining: 2.66s
50:	learn: 6.2459200	total: 921ms	remaining: 2.69s
100:	learn: 6.0770968	total: 1.54s	remaining: 1.51s
150:	learn: 5.8419251	total: 2.21s	remaining: 717ms
199:	learn: 5.6503679	total: 2.82s	remaining: 0us


In [77]:
y_pred = model.predict(test_pool)

# Attach predictions to test set for review
WR_test_pred = X_test.copy()
WR_test_pred['predicted_fantasy_points'] = y_pred

print(WR_test_pred[['season', 'week', 'opponent_team', 'predicted_fantasy_points', 'player_display_name']])

       season  week opponent_team  predicted_fantasy_points  \
16992    2024    14           PHI                 11.476877   
17027    2024    14            SF                 11.976058   
17076    2024    14           LAC                 10.162134   
17120    2024    14           NYJ                  4.483168   
17126    2024    14           CIN                 10.032227   
...       ...   ...           ...                       ...   
22251    2024    14           NYJ                  3.907022   
22285    2024    14           TEN                 14.548807   
22302    2024    14           LAC                  8.407575   
22385    2024    14           CHI                  5.247481   
22424    2024    14            SF                  9.225765   

      player_display_name  
16992        Adam Thielen  
17027        Keenan Allen  
17076     DeAndre Hopkins  
17120       Odell Beckham  
17126       Brandin Cooks  
...                   ...  
22251    Malik Washington  
22285        Brian 

In [78]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Actuals and predictions
y_true = y_test
y_pred = WR_test_pred['predicted_fantasy_points']

# RMSE and MAE
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")

✅ RMSE: 57.93
✅ MAE: 5.60


# **TE CatBoost Model**

In [80]:
df_te= weekly_stats_all[weekly_stats_all['position'] == 'TE'].copy()

# -------------------------------
# STEP 2: Sort for time-series ops
# -------------------------------
df_te = df_te.sort_values(by=['player_id', 'season', 'week'])

# -------------------------------
# STEP 3: Lag + Rolling Features (3-game window)
# -------------------------------
rolling_cols = [
    'fantasy_points_ppr', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_epa', 'receiving_2pt_conversions', 'target_share', 'offense_snaps', 'offense_pct', 'air_yards_share'
    ]

for col in rolling_cols:
    df_te[f'{col}_last3'] = df_te.groupby('player_id')[col].shift(1).rolling(3, min_periods = 1).mean()

# -------------------------------
# STEP 4: Expanding (season-to-date) averages
# -------------------------------
expanding_cols = ['receiving_yards', 'receiving_tds', 'fantasy_points_ppr'
]

for col in expanding_cols:
    df_te[col] = pd.to_numeric(df_te[col], errors='coerce')

for col in expanding_cols:
    df_te[f'{col}_season_avg'] = (
        df_te.groupby(['player_id', 'season'])[col]
        .transform(lambda x: x.shift(1).expanding().mean())
    )


df_te['receiving_yards'] = pd.to_numeric(df_te['receiving_yards'], errors='coerce')
df_te['receptions'] = pd.to_numeric(df_te['receptions'], errors='coerce')


df_te['yards_per_catch'] = df_te['receiving_yards'] / df_te['receptions'].replace(0, pd.NA)
df_te['yards_per_catch'] = pd.to_numeric(df_te['yards_per_catch'], errors='coerce')



print(df_te.head(10))

     player_id player_name player_display_name position position_group  \
54  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
55  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
56  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
57  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
58  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
59  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
60  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
61  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
62  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   
63  00-0024243     M.Lewis      Marcedes Lewis       TE             TE   

                                         headshot_url recent_team  season  \
54  https://static.www.nfl.com/image/private/f_aut...          GB    2021   
55  https://static.www.nfl.com/

In [81]:
# -------------------------------
# STEP 5: Efficiency metrics
# -------------------------------
df_te['carries_shifted'] = df_te.groupby('player_id')['carries'].shift(1)
df_te['receptions_shifted'] = df_te.groupby('player_id')['receptions'].shift(1)


df_te['yards_per_reception_last3'] = df_te.groupby('player_id')['yards_per_catch'].shift(1).rolling(3,min_periods = 1).mean()
df_te['rec_td_rate_last3'] = df_te.groupby('player_id')['receiving_tds'].shift(1).rolling(3,min_periods = 1).sum() / df_rb['receptions_shifted']

# -------------------------------
# STEP 6: Advanced metrics
# -------------------------------
advanced = ['wopr', 'racr']
for col in advanced:
    df_te[f'{col}_trend'] = df_te.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 7: Opponent Defense (already lagged by design)
# -------------------------------
opponent_cols = ['opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg', 'opp_sacks_pg', 'opp_fumbles_pg']
for col in opponent_cols:
    df_te[f'{col}_trend'] = df_te.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

In [82]:
TARGET_COL = 'fantasy_points_ppr'

train_df = df_te[
    (df_te[TARGET_COL].notna()) &
    (
        (df_te['season'] < 2024) |
        ((df_te['season'] == 2024) & (df_te['week'] < 14))
    )
]

test_df = df_te[(df_te['season'] == 2024) & (df_te['week'] == 14)]


feature_cols = [
    col for col in df_te.columns
    if col.endswith('_last3') or col.endswith('_trend') or col.endswith('_season_avg')
] + ['player_display_name','season', 'week', 'opponent_team']


X_train = train_df[feature_cols]
y_train = train_df[TARGET_COL]

X_test = test_df[feature_cols]

# for RMSE testing purposes on the test set
y_test = test_df[TARGET_COL]

In [83]:
categorical_cols = ['opponent_team', 'season', 'week', 'player_display_name']

# Create Pool objects
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_cols)
test_pool = Pool(data=X_test, cat_features=categorical_cols)

# Initialize the CatBoost model
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50  # or 0 to suppress output
)

# Train
model.fit(train_pool)

0:	learn: 5.8870844	total: 11.4ms	remaining: 2.27s
50:	learn: 4.8238439	total: 751ms	remaining: 2.19s
100:	learn: 4.5655891	total: 1.58s	remaining: 1.55s
150:	learn: 4.3502330	total: 2.42s	remaining: 785ms
199:	learn: 4.1631409	total: 3.09s	remaining: 0us


In [84]:
y_pred = model.predict(test_pool)

# Attach predictions to test set for review
TE_test_pred = X_test.copy()
TE_test_pred['predicted_fantasy_points'] = y_pred

print(TE_test_pred[['season', 'week', 'opponent_team', 'predicted_fantasy_points', 'player_display_name']])

       season  week opponent_team  predicted_fantasy_points  \
17057    2024    14           LAC                 13.532261   
17229    2024    14           CLE                  2.958564   
17362    2024    14           JAX                  3.542465   
17585    2024    14           CHI                 10.375439   
17659    2024    14           ARI                  3.149731   
17744    2024    14           CHI                  4.164521   
17843    2024    14           NYJ                  9.456212   
17884    2024    14           TEN                 10.690734   
17894    2024    14           PIT                 15.154769   
18053    2024    14            KC                  5.874594   
18089    2024    14           MIA                  7.099913   
18151    2024    14           PIT                  5.764279   
18254    2024    14           DAL                  5.474168   
18340    2024    14           NYJ                  3.233880   
18372    2024    14           DAL                  8.38

In [86]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Actuals and predictions
y_true = y_test
y_pred = TE_test_pred['predicted_fantasy_points']

# RMSE and MAE
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")

✅ RMSE: 13.69
✅ MAE: 3.01
